In [ ]:
import torch
from torch import nn
import torchvision.models as models

model = models.resnet18(num_classes=26)  #！！！！！！remember to change this to correct number！！！
conv1 = model.conv1
model.conv1 = nn.Conv2d(
  2, conv1.out_channels, conv1.kernel_size, conv1.stride, conv1.padding,
  conv1.dilation, conv1.groups, conv1.bias)

In [ ]:
import random
device = cuda0 = torch.device('cuda:0')
cpu = torch.device('cpu')


features_fn = '/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/GEO_model_training/input/resnet_node_features.txt'
graph_targets_fn = '/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/GEO_model_training/input/resnet_graph_targets.txt'

# magic numbers
INPUT_CHANNELS = 1
OUTPUT_CHANNELS = 26
BATCH_SIZE = 64
EPOCHS = 500 #set this to 200 - 2000
BENCHMARKING = False
random.seed = 88888888

In [ ]:
# we'll need the target encoder from this code block... not sure if we need anything else
def build_resnet_datalist(node_features_fn, graph_targets_fn):
    feature_v = numpy.loadtxt(node_features_fn)
    target_v = numpy.loadtxt(graph_targets_fn,dtype=str,delimiter=",")
    
    target_encoder = sklearn.preprocessing.LabelEncoder()
    target_v = target_encoder.fit_transform(target_v)
    label_mapping = dict(zip(target_encoder.transform(target_encoder.classes_), target_encoder.classes_))
    print(label_mapping)
    print(len(target_v))
    print("----")
    print(len(feature_v))
    data_list = []
    for row_idx in range(len(feature_v)):
      x = torch.tensor(feature_v[row_idx,:],dtype=torch.float)
      x = x.reshape(2,8,491)
      y = torch.tensor([target_v[row_idx]])
      data_list.append({'x':x,'y':y})
    
    return data_list

def build_reactome_graph_loader(data_list,batch_size):

    loader = DataLoader(data_list,batch_size=batch_size,shuffle=True)

    return loader

In [ ]:
import numpy
import sklearn
from sklearn import preprocessing
from torch.utils.data import Dataset, DataLoader
data_list = build_resnet_datalist(features_fn, graph_targets_fn)
random.shuffle(data_list)

In [ ]:
import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters())
criterion = torch.nn.CrossEntropyLoss()

def train(loader,device):
  model.train()

  for batch in loader:  # Iterate in batches over the training dataset.
    x = batch['x'].to(device)
    y = batch['y'].to(device)
    out = model(x)  # Perform a single forward pass.
    y = torch.squeeze(y)
    loss = criterion(out, y)  # Compute the loss.
    loss.backward()  # Derive gradients.
    optimizer.step()  # Update parameters based on gradients.
    optimizer.zero_grad()  # Clear gradients.

def test(loader,device):
  model.eval()

  correct = 0
  for batch in loader:  # Iterate in batches over the training/test dataset.
    x = batch['x'].to(device)
    y = batch['y'].to(device)
    out = model(x)  # Perform a single forward pass.
    y = torch.squeeze(y)
    loss = criterion(out, y)  # Compute the loss.
    pred = out.argmax(dim=1)  # Use the class with highest probability.
    correct += int((pred == y).sum())  # Check against ground-truth labels.
  return correct / len(loader.dataset)  # Derive ratio of correct predictions.

In [ ]:
acc_str = ''
fold = 'full_dataset'
fold_size = 911

#>>> train =              z[:fold_size * (fold - 1)] +         z[fold_size * fold:]
#train_data_list = data_list[:fold_size * (fold - 1)] + data_list[fold_size * fold:]
#>>> test =              z[fold_size * (fold - 1):fold_size * fold]
#test_data_list = data_list[fold_size * (fold - 1):fold_size * fold]
train_data_list = data_list

print(f'Number of training examples: {len(train_data_list)}')
#print(f'Number of test examples: {len(test_data_list)}')
train_data_loader = build_reactome_graph_loader(train_data_list,BATCH_SIZE)
#test_data_loader = build_reactome_graph_loader(test_data_list,BATCH_SIZE)
for epoch in range(EPOCHS):
  train(train_data_loader,device)
  train_acc = test(train_data_loader,device)
  #test_acc = test(test_data_loader,device)
  acc_str += f'{train_acc:.4f}'#',{test_acc:.4f}\n'
  print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}')#', Test Acc: {test_acc:.4f}')

training_acc_fn = F"resnet_classification_acc_fold_{fold}.txt"
path = F"/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/GEO_model_training/GNN/{training_acc_fn}"
with open(path, 'w') as writefile:
    writefile.write(acc_str)
model_save_name = F"trained_pytorch_model_fold_{fold}.pt"
path = F"/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/GEO_model_training/GNN/{model_save_name}" 
torch.save(model.state_dict(), path)
print(F"model saved as {path}")